In [10]:
# export_relative_trajectories_no_likelihood.ipynb 用セル

# ===== 基本設定（Jupyter 互換）=====
import os, sys, re, math, argparse
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 余計なGUIバックエンドを避ける（任意）
plt.switch_backend("Agg")

# --- ユーザーデフォルト（必要に応じて変更） ---
DEFAULT_INPUT  = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv"
DEFAULT_OUTPUT = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\process"
# 研究で使っている5点（tail_set を必ず含める）
DEFAULT_KEYPOINTS = "tail_set,right_tarsal,right_paw,left_tarsal,left_paw"

# ===== ユーティリティ =====
def _norm_name(s: str) -> str:
    """キー名のゆらぎ吸収（空白/アンダースコア/ハイフン除去 + 小文字化）"""
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    """CSV にある実名を、ユーザ指定の論理名から逆引きする"""
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定キーポイントがCSVで見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

def read_dlc_xy_nointerp(csv_path: Path, keypoints: list[str]) -> tuple[np.ndarray, list[str]]:
    """
    DLC(3段ヘッダ)から指定キーポイントの (x,y) を時系列で読み出す。
    - likelihood は一切見ない
    - 欠損補間もしない（そのままの値を返す：NaN があれば NaN のまま）
    返り値:
      X: (T, 2*K) float32  [kp0_x, kp0_y, kp1_x, kp1_y, ...]
      used_kps: 実際に解決した CSV 上のボディパート名（実名）
    """
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    cols = {}
    for bp in use_kps:
        # x, y をそのまま取得（補間なし）
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    X = X_df.values.astype(np.float32)  # (T, 2K) 補間なし
    return X, use_kps

def make_windows(n_frames: int, win_len: int, stride: int):
    """開始インデックスのイテレータを返す（端数切り捨て）"""
    if n_frames < win_len:
        return []
    return list(range(0, n_frames - win_len + 1, stride))

def sanitize(s: str) -> str:
    return re.sub(r'[^a-zA-Z0-9_.-]+', '_', s)

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

# ===== メインロジック =====
def export_trajectories_no_likelihood(
    in_dir: Path,
    out_dir: Path,
    keypoints: list[str],
    window_len: int = 60,
    stride: int = 30,
    invert_y: bool = False,
):
    """
    tail_set を基準に各キーポイントの相対座標軌跡をプロット画像として保存。
    - likelihood 無視、補間なし
    - NaN が含まれるフレームはそのままプロット（線が切れることがあります）
    出力:
      out_dir / <kp_name_sanitized> / <元ファイル名>__w<index>.png
    """
    in_dir  = in_dir.expanduser()
    out_dir = out_dir.expanduser()
    ensure_dir(out_dir)

    csv_paths = sorted([p for p in in_dir.glob("*.csv") if p.is_file()])
    if not csv_paths:
        print(f"[WARN] CSV が見つかりません: {in_dir}")
        return

    # tail_set が含まれていることを確認
    low = [_norm_name(k) for k in keypoints]
    if _norm_name("tail_set") not in low:
        raise ValueError("keypoints に 'tail_set' を必ず含めてください。")

    for csv_path in csv_paths:
        try:
            X, used_kps = read_dlc_xy_nointerp(csv_path, keypoints)
        except Exception as e:
            print(f"[WARN] 読込失敗: {csv_path.name}: {e}")
            continue

        # tail_set のインデックス
        lp = [s.lower() for s in used_kps]
        tidx = lp.index("tail_set")  # 例外になれば except に落ちる

        T = X.shape[0]
        starts = make_windows(T, window_len, stride)
        if not starts:
            print(f"[INFO] {csv_path.name}: フレーム不足でスキップ (T={T} < {window_len})")
            continue

        # キーポイントごとの出力先ディレクトリを準備（tail_set は原点なので除く）
        for i, bp in enumerate(used_kps):
            if i == tidx:
                continue
            ensure_dir(out_dir / sanitize(bp))

        base = sanitize(csv_path.stem)

        # 各ウィンドウで相対座標を計算→プロット
        for wi, s0 in enumerate(starts):
            s1 = s0 + window_len
            # tail_set の (x,y)（補間なし）
            tail_x = X[s0:s1, 2*tidx]
            tail_y = X[s0:s1, 2*tidx + 1]

            # 各キーポイントの相対軌跡
            for i, bp in enumerate(used_kps):
                if i == tidx:
                    continue
                kx = X[s0:s1, 2*i]   - tail_x
                ky = X[s0:s1, 2*i+1] - tail_y
                if invert_y:
                    ky = -ky

                fig = plt.figure(figsize=(5,5))
                ax = fig.add_subplot(111)
                ax.plot(kx, ky, marker="o", linewidth=1)
                ax.set_title(f"{bp} (relative to tail_set)\n{csv_path.name}  frames[{s0}:{s1})")
                ax.set_xlabel("dx [px]")
                ax.set_ylabel("dy [px]")
                ax.axhline(0, alpha=0.3)
                ax.axvline(0, alpha=0.3)
                ax.set_aspect("equal")
                fig.tight_layout()

                out_png = out_dir / sanitize(bp) / f"{base}__w{wi:04d}.png"
                fig.savefig(out_png, dpi=150)
                plt.close(fig)

        print(f"[OK] {csv_path.name}: {len(starts)} ウィンドウ出力")

# ===== CLI / Notebook 互換 =====
def main():
    ap = argparse.ArgumentParser(
        description="Tail-centered relative trajectories exporter (no likelihood, no interpolation)"
    )
    ap.add_argument("--input",  type=str, default=DEFAULT_INPUT,  help="入力CSVフォルダ（DLC 3段ヘッダ）")
    ap.add_argument("--output", type=str, default=DEFAULT_OUTPUT, help="出力フォルダ（各KPごとにサブフォルダ作成）")
    ap.add_argument("--keypoints", type=str, default=DEFAULT_KEYPOINTS,
                    help="カンマ区切りのキーポイント名。必ず tail_set を含めること")
    ap.add_argument("--window-len", type=int, default=60, help="1枚の軌跡に含めるフレーム長")
    ap.add_argument("--stride",     type=int, default=30, help="ウィンドウのストライド")
    ap.add_argument("--invert-y",   action="store_true",  help="y軸を反転（上を正に）")

    # Notebook から渡される未知引数（--f=...json 等）を無視
    if 'ipykernel' in sys.modules:
        args, _ = ap.parse_known_args()
    else:
        args = ap.parse_args()

    in_dir  = Path(args.input)
    out_dir = Path(args.output)
    kps = [s.strip() for s in args.keypoints.split(",") if s.strip()]

    export_trajectories_no_likelihood(
        in_dir=in_dir,
        out_dir=out_dir,
        keypoints=kps,
        window_len=args.window_len,
        stride=args.stride,
        invert_y=args.invert_y,
    )

if __name__ == "__main__":
    main()


[OK] normal_10DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 7 ウィンドウ出力
[OK] normal_11DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 8 ウィンドウ出力
[OK] normal_13DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 5 ウィンドウ出力
[OK] normal_17DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 15 ウィンドウ出力
[OK] normal_20DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 4 ウィンドウ出力
[OK] normal_27DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 8 ウィンドウ出力
[OK] normal_31DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 3 ウィンドウ出力
[OK] normal_33DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 2 ウィンドウ出力
[OK] normal_34DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 4 ウィンドウ出力
[OK] normal_35DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 3 ウィンドウ出力
[OK] normal_37DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 4 ウィンドウ出力
[OK] normal_38DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 7 ウィンドウ出力
[OK] normal_3DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 3 ウィンドウ出力
[OK] normal_40DLC_resnet152_sotuken1Dec17shuffle1_100000.csv: 3 